In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import numpy as np
import time
import re

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
aeropuertos = ["CUN","HAVE","ORD","JFK","LAX","BCN","MAD","YYZ","MCO","SFO","LAS","MUC","YUL","MIA","DEN"] 
urls = []
for x in aeropuertos:
    amx_url = f"https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_{x}_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0"
    urls.append(amx_url)
    print(amx_url)

https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_CUN_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_HAVE_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_ORD_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_JFK_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_LAX_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_BCN_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_MAD_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_YYZ_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_MCO_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-m

In [4]:
amx_data = []

for x in urls:
    browser.visit(x)
    time.sleep(15) #aunque esperes mucho tiempo, dependes de la respuesta del servidor de la página
    try:
        browser.find_by_xpath("/html/body/div[2]/div/div/div[1]/div/div[1]/div/div/button").click()
    except:
        print("ok")
    amx_site = browser.html
    amx_soup = bs(amx_site, 'html.parser')
    options = amx_soup.find_all('div', class_="FlightOptionsListItem") #asegurate de que la ventana sea compacta
    print("appending")
    amx_data.append(options)

appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending


In [5]:
precios = []
origenes = []
destinos = []
salidas = []
llegadas = []

for x in amx_data:
    for y in x:
        precio = y.find('div', class_='FlightOptionsFare-price')
        origen = y.find_all('div', class_='FlightOptionsTimeline-city')
        horario = y.find_all('div', class_='FlightOptionsTimeline-time')
        try:
            precios.append(precio.span.text)
            origenes.append(origen[0].text)
            destinos.append(origen[1].text)
            salidas.append(horario[0])
            llegadas.append(horario[1])
        except:
            continue

In [6]:
salidas2 = []

for x in salidas:
    try:
        salidas2.append(x.text)
    except:
        continue

In [7]:
llegadas2 = []

for x in llegadas:
    try:
        llegadas2.append(x.text)
    except:
        continue

In [8]:
df = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas2,
"Hora Llegada": llegadas2,
"Desde": precios 
})

In [9]:
grouped_df = df.groupby(["Hora Salida"])
amx_grouped = grouped_df.max()

In [10]:
amx_grouped

,Origen,Destino,Hora Llegada,Desde
Hora Salida,,,,
01:15,MEX,YUL,07:25,$4391
06:01,MEX,SFO,19:30,$9471
06:30,MEX,SFO,17:28,$9471
07:00,MEX,SFO,20:10,$9471
07:10,MEX,MIA,11:30,"$10,851"
07:15,MEX,JFK,13:00,$9259
07:20,MEX,SFO,17:38,$9471
07:30,MEX,LAX,22:40,$4824
08:00,MEX,MCO,22:40,$4476


In [11]:
amx_grouped.reset_index(inplace=True)
amx_grouped

,Hora Salida,Origen,Destino,Hora Llegada,Desde
0,01:15,MEX,YUL,07:25,$4391
1,06:01,MEX,SFO,19:30,$9471
2,06:30,MEX,SFO,17:28,$9471
3,07:00,MEX,SFO,20:10,$9471
4,07:10,MEX,MIA,11:30,"$10,851"
5,07:15,MEX,JFK,13:00,$9259
6,07:20,MEX,SFO,17:38,$9471
7,07:30,MEX,LAX,22:40,$4824
8,08:00,MEX,MCO,22:40,$4476
9,08:40,MEX,JFK,21:45,$7878


In [12]:
amx_grouped.to_csv("aeromexico.csv")

In [13]:
aeropuertos = ["CUN","HAVE","ORD","JFK","LAX","BCN","MAD","YYZ","MCO","SFO","LAS","MUC","YUL","MIA","DEN"] 
urls = []
for x in aeropuertos:
    amx_url = f"https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd={x}&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web"
    urls.append(amx_url)
    print(amx_url)

https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=CUN&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=HAVE&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=ORD&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=JFK&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=A

In [14]:
avi_prices = []
avi_horarios = []

for x in urls:
    browser.visit(x)
    time.sleep(15) #aunque esperes mucho tiempo, dependes de la respuesta del servidor de la página
    avi_site = browser.html
    avi_soup = bs(avi_site, 'html.parser')
    avi_prices.append(avi_soup.find_all('div', class_='availability-list-fares'))
    avi_horarios.append(avi_soup.find_all('div', class_='flight-details availability-flight-details flight-details-without-button availability-flight-details-without-button row'))
    print("appending")

appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending


In [15]:
precios = []

for x in avi_prices:
    for y in x:
        precios.append(y.find('div', class_='cell-reco-price').span.text)

In [16]:
salidas = []
llegadas = []
origenes = []
destinos = []

for x in avi_horarios:
    for y in x:
        salidas.append(y.find('time', class_='time-from').text)
        llegadas.append(y.find('time', class_='time-to').text)
        origenes.append(y.find('abbr', class_='citycode-from').text.replace("(","").replace(")",""))
        destinos.append(y.find('abbr', class_='citycode-to').text.replace("(","").replace(")",""))

In [18]:
df2 = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas,
"Hora Llegada": llegadas,
"Desde": precios
})

In [ ]:
df2.to_csv("avianca.csv")

In [ ]:
aeropuertos = ["JFK","LAX","MIA"]
urls = []
for x in aeropuertos:
    ua_url = f"https://www.united.com/ual/en/MX/flight-search/book-a-flight/results/?f=MEX&t={x}&d=2019-11-15&tt=1&sc=7&px=1&taxng=1&newHP=True&idx=1"
    urls.append(ua_url)
    print(ua_url)

In [ ]:
ua_data = []

for x in urls:
    browser.visit(x)
    time.sleep(5)
    browser.visit(x)
    time.sleep(15)
    united_site = browser.html
    united_soup = bs(united_site, 'html.parser')
    print("appending")
    ua_data.append(united_soup.find_all('li', class_='flight-block flight-block-fares use-roundtrippricing flight-block-revised'))

In [ ]:
ua_data[2]

In [ ]:
salidas = []
llegadas = []
origenes = []
destinos = []
precios = []

for x in ua_data:
    for y in x:
        xx = y.find('div', class_='flight-time flight-time-depart').text
        xy = y.find('div', class_='flight-time flight-time-arrive').text
        xz = y.find_all('span')
        xw = (y.find('div', class_='price-point price-point-revised use-roundtrippricing').text)
        salidas.append(re.search(r'\d{1,2}(:\d{1,2})', xx).group())
        llegadas.append(re.search(r'\d{1,2}(:\d{1,2})', xy).group())
        origenes.append(xz[1].text)
        destinos.append(xz[3].text)
        precios.append(xw.replace("\n", "").replace("US", "").replace("$",""))

In [ ]:
df3 = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas,
"Hora Llegada": llegadas,
"Desde": precios
})
df3["Origen"] = df3["Origen"].str[-4:]
df3["Destino"] = df3["Destino"].str[-4:]
df3["Origen"] = df3["Origen"].str[:3]
df3["Destino"] = df3["Destino"].str[:3]
df3.drop(37)
df3

In [ ]:
df3.to_csv("united.csv")